# The method


The method that will be described in this section was inspired by a feature extraction technique used in image processing - the Hough Transform. The original Hough transform was developed to detect lines in an image, in order to support the detection of charged particles using bubble chambers {cite}`hough`. Since then, it has been generalized to detect more shapes and has become a standard method in image processing and feature extraction.
```{margin}
The Hough Transform is a feature extraction technique used in image analysis to find imperfect instances of shapes by a voting procedure in parameter space, from which object candidates are obtained as local maxima in a so-called accumulator space.
```

The generalized Hough transform, normally needs an analytical function that can be discretized in order to transform from image to parameter space and vise-versa. As we will see below, deriving an analytical solution for the projection of complex 3D surfaces is not possible in most cases. Here, we develop a Hough-like voting method, by introducing a numerical approach to this projection problem.

We will demonstrate the methodology using the simplest empirical model, the Shue model. We generate a set of Shue surfaces for a range of the model's parameters, $r_0$ and $\alpha$. To do this, we convert the Shue model to a 3D surface in spherical coordinates by rotating over the symmetry axis (x) with the $\phi$ angle. Due to its symmetrical nature, this results in the same expression:

$$r(\theta,\phi) = r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha$$

For each of these surfaces we compute the tangent point to the satellite through a semi-analytical approach. We then convert to cartesian coordinates, thus parametrizing the surface with respect to $\theta$ and $\phi$. 

$$ x = r cos\theta = r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha \cos\theta$$
$$y = r sin\theta cos \phi = r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha \sin \theta \cos \phi $$
$$ z = r sin\theta sin\phi = r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha \sin \theta \sin \phi $$

Or in parametrized vector form:

$$
R(\theta,\phi) &= \left( r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha \cos\theta \right) \hat{i} \quad + \left( r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha \sin \theta \cos \phi \right) \hat{j} \\
&\quad + \left( r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha \sin \theta \sin \phi \right) \hat{k}
$$

Having a parametrized surface, we can compute the normal vector function for each point, through the cross product of the partial derivative of this parametrized function, with respect to each parameter.

$$
R_{\theta} = \frac{\partial R}{\partial \theta}= r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha &\Bigg[
\sin \theta \frac{(a - 1) \cos \theta - 1}{\cos \theta + 1} \, \hat{i} \\
&\quad + \cos \phi \left( \cos \theta + \frac{\sin^2 \theta \, a}{1 + \cos \theta} \right) \, \hat{j} \\
&\quad + \sin \phi \left( \cos \theta + \frac{\sin^2 \theta \, a}{1 + \cos \theta} \right) \, \hat{k}
\Bigg]
$$

$$R_{\phi} = \frac{\partial R}{\partial \phi } = r_0 \left( \frac{2}{1 + \cos \theta} \right)^\alpha \sin \theta\left( - \sin \phi \hat{j} + \cos \phi\hat{k}\right) $$

Therefore, the normal vector function can be expressed as in Equation {eq}`eq:normal`

$$
    \eta = R_{\theta} \times R_{\phi} = r_{shue}^2 \sin \theta \left [\left( \cos \theta + \frac{\sin^2 \theta a}{1 + \cos \theta } \right)(\cos^2 \phi +\sin^2 \phi) \hat{i} \right] + \dots \hat{j} + \dots \hat{k}
$$(eq:normal)

We can now find the tangent direction in the FOV of the satellite by requesting that the normal vector and the vector that connects each point of the surface to the satellite, are perpendicular to each other. This gives us a set of $(\theta,\phi)$ - a curve in 3D space. The tangent curve is given in GSE coordinates by Equation {eq}`eq:tangent`, when solving for $(x, y, z)$.

$$
    \hat{ \eta } \cdot \vec{ r_{sat}} = 0 \implies (x_{sat}-x) r_0^2 \left( \frac{2}{1 + \cos \theta} \right)^{2\alpha} \sin \theta \left( \cos \theta + \frac{\sin^2 \theta a}{1 + \cos \theta } \right) + \dots  = 0
$$(eq:tangent)


This is unsolvable analytically, or with series expansion, and therefore requires that we solve it numerically for each curve and each satellite \ac{POV}. It would be valuable to compare this with a fully numerical calculation of the normal vector function, as described in \autoref{numerical_surface_projection}, and optimize the projection process. In both cases the result is a set of points in the \ac{GSE} frame. By applying a coordinate transformation from GSE to the SXI frame, as explained in \autoref{numerical_surface_projection}, we get a set of discretized curves, that correspond to the set of initial conditions of the Shue models. We interpolate the curves inside the FOV and rasterize them to the pixel grid of the image. A visualization of this process is shown in \autoref{fig:tangent_points_projection}.


```{figure} ./images/Hough/method/tangent.png
:align: right
Tangent points of Shue surface in SXI image frame.
```

For each curve in the image coordinates, we calculate the intensity of the image along the curve and save its mean value in an array element. This gives us a measure of how prominent and bright this curve is in the image. We can now use this metric as a voting scheme between the curves by moving to the parameter space. Each curve and mean intensity correspond to specific initial parameters $\alpha$ and $r_0$. We can construct a parameter matrix with the values of the mean intensity over each curve. The visualization of this matrix in a contour plot is shown in \autoref{fig:hough_matrix}. In this figure, the maximum value in parameter space is noted with the red dot. The coordinates of this point, correspond to the model parameters that return the brightest projection in the image. Moving back to image space, we plot only the curve that correspond to these parameters in \autoref{fig:fitted_curve}, denoted with the blue line.
